In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from bs4 import BeautifulSoup
import requests
import json
from IPython.display import Markdown
from IPython.display import update_display


In [ ]:
load_dotenv(override="true")
api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

In [ ]:
openai = OpenAI()
MODAL='gpt-5-nano'

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


In [ ]:
link_system_prompt = """
You are a UX designer.
You are given a website and you look at it and find ways to improve how the UX could look better.
You should look at options like making the website more modern and catchy.
It could br usage of better css or animations or the way different components are rendered
"""

In [ ]:
def get_user_prompt(url):
    user_prompt = f"""
    here is a link to the website {url} -
    Please provide options to make this website better
     """
    return user_prompt

In [ ]:
print(get_user_prompt('https://www.google.com'))

In [ ]:
def suggest_better_design_options(url):
    response = openai.chat.completions.create(
        model=MODAL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": 'user', "content": get_user_prompt(url)}
        ]
    )
    result = response.choices[0].message.content
    return result

In [ ]:
suggest_better_design_options('https://www.google.com')

In [ ]:
def make_better_UX(url):
    contents = suggest_better_design_options(url)
    result = f"## Suggested Changes:\n\n{contents}\n##:\n"
    return result

In [ ]:
print(make_better_UX("https://www.google.com"))

In [ ]:
ux_desing_system_prompt = """
You are an assistant that generates a webpage sample using provided suggestions.
dont give me code, but a sample design
"""

In [ ]:
def get_ux_user_prompt(url):
    user_prompt = f"""
    You are looking at following suggestions for this website: {url}
    """
    user_prompt = make_better_UX(url)[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_ux_user_prompt('https://www.google.com')

In [ ]:

def create_better_ux(url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": 'system', 'content': ux_desing_system_prompt},
            {'role': 'user', 'content': get_ux_user_prompt(url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_better_ux('https://www.google.com')